In [1]:
import os
import pandas as pd
import numpy as np
import pymc3 as pm

In [2]:
base_dir = os.path.join("/Users", "sbussmann", "Development", "buda", "buda-ratings")
interim_dir = os.path.join(base_dir, 'data', 'interim')

In [3]:
league_id = 40264
winloss = pd.read_csv(os.path.join(interim_dir, "winloss_simpleprior_{}.csv".format(league_id)))

In [4]:
winloss.head(20)

,Team A,Team B,Team A Wins,Index A,Index B
0,AHOC,Gothrilla,True,0,1
1,AHOC,BBN,True,0,9
2,AHOC,Stonecutters,True,0,41
3,AHOC,FlowChart,True,0,2
4,AHOC,Lady and the BAMF,True,0,28
5,AHOC,Swingers,True,0,5
6,AHOC,Upstream,True,0,10
7,AHOC,Tubbs,True,0,29
8,AHOC,Stonecutters,True,0,41
9,AHOC,Zerg Rush!,True,0,4


In [5]:
winloss.shape

(522, 5)

In [6]:
teams = set(np.append(winloss['Team A'].unique(), winloss['Team B'].unique()))

In [17]:
team1 = winloss['Index A'].values
team2 = winloss['Index B'].values

with pm.Model() as model:
    
    ratings = pm.Normal('ratings', mu=0, shape=len(teams))
    
    deltaRating = ratings[team1] - ratings[team2]

    p = 1 / (1 + np.exp(-deltaRating))
    win = pm.Bernoulli('win', p, observed=winloss['Team A Wins'].values)

In [18]:
with model:
    trace = pm.sample(1000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [ratings]
100%|██████████| 1500/1500 [00:02<00:00, 528.58it/s]


In [19]:
meanratings = trace.get_values('ratings').mean(axis=0)

In [11]:
names = []
for i, meanrating in enumerate(meanratings):
    if i in winloss['Index A'].values:
        index = winloss['Index A'] == i
        name = winloss.loc[index, 'Team A'].unique()[0]
    else:
        index = winloss['Index B'] == i
        name = winloss.loc[index, 'Team B'].unique()[0]
    names.append(name)
#     print("{}: {:.3f}".format(name, meanrating))

In [12]:
ratingsdf = pd.DataFrame({
    'mean_rating': meanratings
}, index=names)

In [14]:
plusminus = pd.read_csv(os.path.join(interim_dir, 'plusminus_{}.csv'.format(league_id)), index_col='Team Name')

In [15]:
totaldf = plusminus.join(ratingsdf, how='inner')

In [16]:
totaldf.sort_values('mean_rating', ascending=False)

,PlusMinus,divname,Wins,Losses,Ties,mean_rating
AHOC,100,4/3 Div 1,14,0,0,2.390247
Injustice League,100,4/3 Div 2,14,1,0,1.743978
Pink Flamingos,58,5/2 Div 2,10,1,0,1.605038
SnakeCountryBromance,33,5/2 Div 1,5,0,0,1.582016
Jack's Abby HAOS Lager,46,4/3 Div 2,10,2,0,1.543060
Maverick,42,5/2 Div 2,9,2,1,1.423889
JuJu Hex,72,5/2 Div 2,11,3,1,1.396716
Zerg Rush!,22,4/3 Div 1,10,4,0,1.338057
Upstream,119,4/3 Div 2,20,6,0,1.329346
Puddingstone,98,4/3 Div 2,18,6,0,1.246406
